<a href="https://colab.research.google.com/github/AliceDragonfly/HomeworkHSE/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

## pandas + парсинг

Здесь находится главная страница [Системного блока](https://sysblok.ru)

Соберите корпус новостей (можно спарсить все новости или небольшую часть)

Дедлайн: 22 декабря включительно

In [ ]:
# необходимые импорты
import requests
from bs4 import BeautifulSoup
import pandas as pd

from tqdm import tqdm

### шаг 1

In [ ]:
# Функция для сбора всех ссылок на страницы с новостями
def collect_news_pages(base_url, last_page):
    # Создание списка URL для всех страниц новостей
    all_pages = [f"{base_url}/page/{i}" for i in range(1, last_page + 1)]
    all_pages.insert(0, base_url)  # Добавление главной страницы
    return all_pages

### шаг 2

In [ ]:
# Функция для сбора всех ссылок на отдельные новости с каждой страницы
def collect_news_links(pages):
    all_links = []
    for page in pages:
        response = requests.get(page)  # Запрос к странице
        soup = BeautifulSoup(response.content, 'html.parser')  # Парсинг содержимого страницы
        for link in soup.find_all('h2'):  # Поиск всех тегов h2, содержащих ссылки на новости
            a_tag = link.find('a')  # Поиск тега a внутри h2
            if a_tag and 'href' in a_tag.attrs:  # Проверка наличия тега a и атрибута href
                all_links.append(a_tag.attrs['href'])  # Добавление ссылки на новость
    return all_links

### шаг 3

работаем с 1 новостью:

нас интересует название, автор, дата публикации, текст (дополнительно можно спарсить тематические категории)

In [ ]:
# Функция для парсинга деталей новости
def parse_news_details(news_url):
    response = requests.get(news_url)  # Запрос к странице новости
    soup = BeautifulSoup(response.content, 'html.parser')  # Парсинг содержимого страницы новости

    # Извлечение нужных данных
    title = soup.find('h1').text.strip()  # Заголовок новости
    date = soup.find('time').text.strip()  # Дата публикации
    author = soup.find('meta', {'name': 'author'})['content'] if soup.find('meta', {'name': 'author'}) else None  # Автор
    text = ' '.join([p.text for p in soup.find_all('p')])  # Текст новости
    categories = [tag.text for tag in soup.find_all('a', rel='category tag')]  # Категории новости

    return {
        'title': title,
        'date': date,
        'author': author,
        'text': text,
        'categories': categories
    }

### шаг 4

In [ ]:
# Основной код для сбора данных
def collect_news_data(base_url, last_page):
    pages = collect_news_pages(base_url, last_page)  # Получение всех страниц новостей
    news_links = collect_news_links(pages)  # Получение всех ссылок на новости
    news_data = []

    for link in tqdm(news_links):
        try:
            news_details = parse_news_details(link)  # Парсинг деталей каждой новости
            news_data.append(news_details)
        except Exception as e:
            print(f"Error parsing {link}: {e}")

    return pd.DataFrame(news_data)  # Преобразование данных в DataFrame

### шаг 5

In [ ]:
# Вызов функции сбора данных

base_url = 'https://sysblok.ru'
last_page = 6  # Примерное количество страниц
news_df = collect_news_data(base_url, last_page)  # Сбор данных

100%|██████████| 426/426 [08:21<00:00,  1.18s/it]


In [ ]:
news_df.sample(n=5) # Пример случайных 5 новостей из набора

,title,date,author,text,categories
256,Мозг меланхолика: видно ли темперамент на ЭЭГ,07.04.2022,Алексеева Екатерина,\nМы часто слышим фразы вроде «Он не проявля...,[Психология]
77,Эйджинг и социальное неравенство: искусственны...,26.11.2023,Ирина Дружинина,"\nООН прогнозирует, что к 2100 году пожилые ...",[Общество]
205,Статистика империи инков: студент дешифровал у...,16.09.2022,Ольга Шелухина,\nСтудент Гарварда сравнил документ переписи...,[История]
176,«Персоналистские диктатуры чаще начинают войны...,10.01.2023,Системный Блокъ,\nСистемный Блокъ поговорил с политологом Эр...,"[Интервью, Общество]"
283,Рекомендательные системы плохо работают для му...,10.01.2022,Мария Адзхед,"\nЕсли вы слушаете не мейнстрим, скорее всег...",[Новости]


In [ ]:
news_df.to_csv('news.csv', index=False)

Готово!
Загрузите ваш код и собранные новости на GitHub